In [1]:
%run input/Format.ipynb
import ROOT as root
import numpy as np
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
Save_to_html = False

In [3]:
N_sec = 8
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="input/"
file_names=["../../VTXalignment/input/my-test_phiep_res_v08.root"]
sim_file_path="../sim/output/Newembed/sngl/"
dalitz_file_names=['../../summer25/electron/dalitz_v20.root']
sim_file_names = []#[dalitz_file_names[0]]
legend_names = ["pos","neg","sim_pos","sim_neg"] 
hist_select_3D_names = [f"emcal_sec{i}" for i in range(N_sec)]
hist_select_3D_names = [f"emcal_hist_{i}" for i in range(N_sec)]

In [4]:
N_hists = len(hist_select_3D_names)

In [5]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    #print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


for iFile in range(len(sim_file_names)):
    #print(sim_file_path+sim_file_names[iFile])
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)

infile.Close()

In [6]:
pt_min, pt_max = 0, 6
iFile = 0
type_loc = 1
c0 = root.TCanvas(f"c0",f"c0",1450,2*900)
c0.Divide(3,3)
projz = hists_read[iFile][0].ProjectionZ(f"projz{type_loc}")
legends = []
emcal_pixel_maps = []
for isector in range(len(hists_read[iFile])):
    c0.cd(1+isector)
    emcal_pixel_maps.append(root.TH2D(f"new_hist_{isector}",f"new_hist_{isector};z;y",hists_read[iFile][isector].GetNbinsX(), hists_read[iFile][isector].GetXaxis().GetBinLowEdge(1),\
                                hists_read[iFile][isector].GetXaxis().GetBinUpEdge(hists_read[iFile][isector].GetNbinsX()),hists_read[iFile][isector].GetNbinsY(),\
                                hists_read[iFile][isector].GetYaxis().GetBinLowEdge(1),hists_read[iFile][isector].GetYaxis().GetBinUpEdge(hists_read[iFile][isector].GetNbinsY())))
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    for ibinz in range(ibinz0,1+ibinz1):
        for ibinx in range(1,1+hists_read[iFile][isector].GetNbinsX()):
            for ibiny in range(1,1+hists_read[iFile][isector].GetNbinsY()):
                emcal_pixel_maps[-1].SetBinContent(ibinx,ibiny,emcal_pixel_maps[-1].GetBinContent(ibinx,ibiny)+hists_read[iFile][isector].GetBinContent(ibinx,ibiny,ibinz))


    emcal_pixel_maps[-1].Draw("colz")

c0.Draw()

In [7]:
def find_hot_pixels(h2,
                    half_win=3,        # neighborhood half-size (3 -> 7x7 window)
                    nsig=5.0,          # threshold in robust sigmas
                    min_neighbors=10,  # minimum valid neighbors to trust stats
                    min_excess=5.0,    # absolute excess above local median
                    zero_cut=0.0,      # neighbors <= zero_cut are treated as dead/cold
                    make_zmap=True):   # build a TH2D with robust z-scores
    nx, ny = h2.GetNbinsX(), h2.GetNbinsY()
    xax, yax = h2.GetXaxis(), h2.GetYaxis()

    zmap = None
    if make_zmap:
        zmap = h2.Clone(h2.GetName()+"_zmap")
        zmap.Reset()
        zmap.SetTitle(h2.GetTitle()+";"+xax.GetTitle()+";"+yax.GetTitle()+";robust z")

    hot_xy = []
    for ix in range(1, nx+1):
        for iy in range(1, ny+1):
            c = h2.GetBinContent(ix, iy)

            # build neighborhood (exclude center and dead/cold neighbors)
            nb = []
            for dx in range(-half_win, half_win+1):
                jx = ix + dx
                if jx < 1 or jx > nx: 
                    continue
                for dy in range(-half_win, half_win+1):
                    jy = iy + dy
                    if jy < 1 or jy > ny or (dx == 0 and dy == 0):
                        continue
                    v = h2.GetBinContent(jx, jy)
                    if v > zero_cut:
                        nb.append(v)

            if len(nb) < min_neighbors:
                if zmap: zmap.SetBinContent(ix, iy, 0.0)
                continue

            nb = np.asarray(nb, dtype=float)
            med = np.median(nb)
            mad = np.median(np.abs(nb - med))
            sigma = 1.4826*mad  # MAD -> sigma (Gaussian)
            # floor sigma to something sensible (helps tiny-stat regions)
            if sigma < 1.0:
                sigma = max(1.0, np.sqrt(max(med, 1.0)))

            z = (c - med)/sigma if sigma > 0 else 0.0
            if zmap: zmap.SetBinContent(ix, iy, z)

            if (c - med) >= min_excess and z >= nsig:
                x = xax.GetBinCenter(ix)
                y = yax.GetBinCenter(iy)
                hot_xy.append((x, y, c, med, z))

    # overlay as a TGraph
    g = root.TGraph(len(hot_xy))
    for i, (x, y, c, med, z) in enumerate(hot_xy):
        g.SetPoint(i, x, y)
    g.SetName(h2.GetName()+"_hotpixels")
    g.SetTitle("Hot pixels")
    g.SetMarkerStyle(31)   # star
    g.SetMarkerSize(1.6)
    g.SetMarkerColor(root.kRed+1)

    return g, zmap, hot_xy


In [8]:
# --- assumes you already filled emcal_pixel_maps[sector] as in your code ---
# and you have the find_hot_pixels() from earlier.
# If you don't, paste that function first.

def _hot_bin_indices(h2, **kwargs):
    """Return list of (ix0, iy0) zero-based hot bin indices using the robust detector."""
    # we call the robust detector but we only need indices; avoid building the zmap
    nx, ny = h2.GetNbinsX(), h2.GetNbinsY()
    # local copy of the detector core (small inline to avoid dependency)
    half_win = kwargs.get("half_win", 3)
    nsig = kwargs.get("nsig", 5.0)
    min_neighbors = kwargs.get("min_neighbors", 10)
    min_excess = kwargs.get("min_excess", 5.0)
    zero_cut = kwargs.get("zero_cut", 0.0)

    hot_idx = []
    for ix in range(1, nx+1):
        for iy in range(1, ny+1):
            c = h2.GetBinContent(ix, iy)
            nb = []
            for dx in range(-half_win, half_win+1):
                jx = ix + dx
                if jx < 1 or jx > nx: continue
                for dy in range(-half_win, half_win+1):
                    jy = iy + dy
                    if jy < 1 or jy > ny: continue
                    if dx==0 and dy==0: continue
                    v = h2.GetBinContent(jx, jy)
                    if v > zero_cut: nb.append(v)
            if len(nb) < min_neighbors: continue
            nb = np.asarray(nb, float)
            med = float(np.median(nb))
            mad = float(np.median(np.abs(nb - med)))
            sigma = 1.4826*mad
            if sigma < 1.0:
                sigma = max(1.0, np.sqrt(max(med, 1.0)))
            z = (c - med)/sigma if sigma>0 else 0.0
            if (c - med) >= min_excess and z >= nsig:
                hot_idx.append((ix-1, iy-1))  # zero-based
    return hot_idx

def write_hot_txt(emcal_pixel_maps, txt_path="hot_pixels.txt",
                  sectors=None, **detector_kwargs):
    """
    Write lines: sector  iy  iz  isdead
    Note: iz corresponds to X (z-axis) bin index; iy corresponds to Y bin index.
    """
    if sectors is None:
        sectors = range(len(emcal_pixel_maps))
    rows = []
    for sec in sectors:
        h2 = emcal_pixel_maps[sec]
        hot_idx = _hot_bin_indices(h2, **detector_kwargs)
        for (ix0, iy0) in hot_idx:
            iz0 = ix0
            rows.append((sec, iy0, iz0, 1))  # isdead==1 for flagged pixel
    rows.sort()
    with open(txt_path, "w") as f:
        for sec, iy0, iz0, flag in rows:
            f.write(f"{sec} {iy0} {iz0} {flag}\n")
    return rows  # useful for the header writer



In [9]:
pt_min, pt_max = 0, 6
iFile = 0
type_loc = 1

c0 = root.TCanvas("c0","c0",1450,2*900)
c0.Divide(3,3)

projz = hists_read[iFile][0].ProjectionZ(f"projz{type_loc}")
emcal_pixel_maps = []
hot_overlays = []
z_maps = []

for isector in range(len(hists_read[iFile])):
    c0.cd(1+isector)

    src = hists_read[iFile][isector]
    h2 = root.TH2D(f"new_hist_{isector}", f"new_hist_{isector};{src.GetXaxis().GetTitle()};{src.GetYaxis().GetTitle()}",
                   src.GetNbinsX(), src.GetXaxis().GetBinLowEdge(1), src.GetXaxis().GetBinUpEdge(src.GetNbinsX()),
                   src.GetNbinsY(), src.GetYaxis().GetBinLowEdge(1), src.GetYaxis().GetBinUpEdge(src.GetNbinsY()))
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    for ibinz in range(ibinz0, ibinz1+1):
        for ibinx in range(1, src.GetNbinsX()+1):
            for ibiny in range(1, src.GetNbinsY()+1):
                h2.SetBinContent(ibinx,ibiny,h2.GetBinContent(ibinx,ibiny)+src.GetBinContent(ibinx,ibiny,ibinz))

    emcal_pixel_maps.append(h2)

    # --- detect hot pixels (tune params if needed) ---
    g, zmap, hot_list = find_hot_pixels(h2,
                                        half_win=3,    # 7x7 local window
                                        nsig=5.0,      # ~5σ above local bg
                                        min_neighbors=10,
                                        min_excess=5.0,
                                        zero_cut=0.0,
                                        make_zmap=True)
    hot_overlays.append(g)
    z_maps.append(zmap)

    # draw base map + overlay markers
    emcal_pixel_maps[-1].Draw("COLZ")
    if g and g.GetN() > 0:
        g.Draw("P SAME")

c0.Draw()

# If you want to list them:
for isector, hot_list in enumerate([hl for (_,_,hl) in [find_hot_pixels(h2, make_zmap=False) for h2 in emcal_pixel_maps]]):
    if not hot_list: 
        continue
    print(f"Sector {isector}: {len(hot_list)} hot pixels")
    for (x,y,c,med,z) in hot_list:
        print(f"  x={x:.2f}, y={y:.2f}, content={c:.1f}, local_med={med:.1f}, z={z:.2f}")


Sector 0: 1 hot pixels
  x=20.50, y=0.50, content=71.0, local_med=39.0, z=5.40
Sector 2: 3 hot pixels
  x=50.50, y=35.50, content=70.0, local_med=46.0, z=5.40
  x=63.50, y=35.50, content=97.0, local_med=44.0, z=5.11
  x=67.50, y=35.50, content=134.0, local_med=38.0, z=6.48
Sector 3: 20 hot pixels
  x=8.50, y=32.50, content=272.0, local_med=40.0, z=11.18
  x=8.50, y=34.50, content=391.0, local_med=37.0, z=15.92
  x=8.50, y=35.50, content=418.0, local_med=34.5, z=19.90
  x=9.50, y=31.50, content=399.0, local_med=41.0, z=20.12
  x=9.50, y=32.50, content=647.0, local_med=40.0, z=29.24
  x=9.50, y=33.50, content=431.0, local_med=38.0, z=15.15
  x=9.50, y=34.50, content=339.0, local_med=37.0, z=10.72
  x=9.50, y=35.50, content=462.0, local_med=38.0, z=14.67
  x=13.50, y=32.50, content=515.0, local_med=38.0, z=40.22
  x=13.50, y=33.50, content=824.0, local_med=38.0, z=66.27
  x=49.50, y=0.50, content=191.0, local_med=49.0, z=13.68
  x=50.50, y=0.50, content=172.0, local_med=52.0, z=16.19
  x=

In [10]:
rows = write_hot_txt(
    emcal_pixel_maps,
    txt_path="output/hot_pixels.txt",
    half_win=3, nsig=5.0, min_neighbors=10, min_excess=5.0, zero_cut=0.0
)

In [11]:
import os

# --- bring in your robust detector from earlier (or paste the same function here) ---
def find_hot_pixels(h2, half_win=3, nsig=5.0, min_neighbors=10, min_excess=5.0, zero_cut=0.0, make_zmap=False):
    nx, ny = h2.GetNbinsX(), h2.GetNbinsY()
    xax, yax = h2.GetXaxis(), h2.GetYaxis()
    hot_xy = []
    for ix in range(1, nx+1):
        for iy in range(1, ny+1):
            c = h2.GetBinContent(ix, iy)
            nb = []
            for dx in range(-half_win, half_win+1):
                jx = ix + dx
                if jx < 1 or jx > nx: continue
                for dy in range(-half_win, half_win+1):
                    jy = iy + dy
                    if jy < 1 or jy > ny: continue
                    if dx==0 and dy==0: continue
                    v = h2.GetBinContent(jx, jy)
                    if v > zero_cut: nb.append(v)
            if len(nb) < min_neighbors: continue
            nb = np.asarray(nb, float)
            med = float(np.median(nb))
            mad = float(np.median(np.abs(nb - med)))
            sigma = 1.4826*mad
            if sigma < 1.0:
                sigma = max(1.0, np.sqrt(max(med, 1.0)))
            z = (c - med)/sigma if sigma>0 else 0.0
            if (c - med) >= min_excess and z >= nsig:
                x = xax.GetBinCenter(ix)
                y = yax.GetBinCenter(iy)
                hot_xy.append((ix-1, iy-1, x, y, c, med, z))  # store 0-based indices too
    g = root.TGraph(len(hot_xy))
    for i, (_, _, x, y, _, _, _) in enumerate(hot_xy):
        g.SetPoint(i, x, y)
    g.SetName(h2.GetName()+"_hotpixels_new")
    g.SetMarkerStyle(29); g.SetMarkerSize(1.6); g.SetMarkerColor(root.kRed+1)
    return g, hot_xy

# --- I/O helpers for your TXT format ---
def load_hot_txt(path):
    """
    Returns dict: sec -> set((iy, iz))
    """
    hot = {}
    if not os.path.exists(path):
        return hot
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"): 
                continue
            parts = line.split()
            if len(parts) < 4: 
                continue
            s, iy, iz, flag = map(int, parts[:4])
            if flag != 1: 
                continue
            hot.setdefault(s, set()).add((iy, iz))
    return hot

def write_hot_txt(path, hot_dict):
    """
    Rewrites the file sorted: sector iy iz 1
    """
    rows = []
    for s, pairs in hot_dict.items():
        for (iy, iz) in pairs:
            rows.append((s, iy, iz))
    rows.sort()
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w") as f:
        for s, iy, iz in rows:
            f.write(f"{s} {iy} {iz} 1\n")
    return len(rows)

# --- mask a TH2 by zeroing flagged bins (does not modify original) ---
def masked_clone(h2, sector, hot_dict, name_suffix="_masked"):
    h = h2.Clone(h2.GetName()+name_suffix)
    h.SetDirectory(0)
    nx, ny = h.GetNbinsX(), h.GetNbinsY()
    flagged = hot_dict.get(sector, set())
    if not flagged:
        return h
    for ix0 in range(nx):     # 0-based for our map (iz)
        for iy0 in range(ny): # 0-based for our map (iy)
            if (iy0, ix0) in flagged:
                # ROOT bins are 1-based:
                h.SetBinContent(ix0+1, iy0+1, 0.0)
                h.SetBinError(ix0+1, iy0+1, 0.0)
    return h

# --- main test/update routine ---
def test_and_update_hot_map(emcal_pixel_maps, txt_path="output/hot_pixels.txt",
                            half_win=3, nsig=5.0, min_neighbors=10, min_excess=5.0, zero_cut=0.0):
    """
    1) Read existing hot map
    2) Build masked clones per sector
    3) Re-run detection; collect only *new* hot pixels
    4) Update TXT (deduped + sorted)
    5) Plot masked maps with new hot overlays
    """
    # 1) read current map
    hot_dict = load_hot_txt(txt_path)

    # 2-3) run per sector
    c = root.TCanvas("c_masked_hot", "Masked maps + NEW hot pixels", 1450, 900*2)
    nsec = len(emcal_pixel_maps)
    cols = 3
    rows = int(np.ceil(nsec/float(cols)))
    c.Divide(cols, rows)
    info = []
    info.append(c)
    any_new = False
    for s in range(nsec):
        c.cd(1+s)
        masked = masked_clone(emcal_pixel_maps[s], s, hot_dict)
        masked.Draw("COLZ")
        g, hot_list = find_hot_pixels(masked, half_win=half_win, nsig=nsig,
                                      min_neighbors=min_neighbors, min_excess=min_excess,
                                      zero_cut=zero_cut, make_zmap=False)
        # Filter out already-known pixels
        new_pairs = []
        known = hot_dict.get(s, set())
        for (ix0, iy0, x, y, cval, med, z) in hot_list:
            iy0_z = iy0
            iz0_z = ix0
            if (iy0_z, iz0_z) not in known:
                new_pairs.append((iy0_z, iz0_z))
        if new_pairs:
            any_new = True
            # overlay only the new ones (optional: hide g and draw a reduced graph)
            g.Draw("P SAME")
            # update dict
            hot_dict.setdefault(s, set()).update(new_pairs)
        info.append(g)
        info.append(masked)
    c.Update()

    # 4) write back if needed
    if any_new:
        nrows = write_hot_txt(txt_path, hot_dict)
        print(f"Updated {txt_path} with new hot pixels. Total lines: {nrows}")
    else:
        print("No new hot pixels found. Map unchanged.")

    return any_new, info

# ----- Example call (after you have emcal_pixel_maps built) -----
# any_new = test_and_update_hot_map(
#     emcal_pixel_maps,
#     txt_path="output/hot_pixels.txt",
#     half_win=3, nsig=5.0, min_neighbors=10, min_excess=5.0, zero_cut=0.0
# )


In [12]:
pt_min, pt_max = 0, 6

projz = hists_read[iFile][0].ProjectionZ(f"projaz{type_loc}")
emcal_pixel_maps = []

for isector in range(len(hists_read[iFile])):
    c0.cd(1+isector)

    src = hists_read[iFile][isector]
    h2 = root.TH2D(f"new_hist_{isector}", f"new_hist_{isector};{src.GetXaxis().GetTitle()};{src.GetYaxis().GetTitle()}",
                   src.GetNbinsX(), src.GetXaxis().GetBinLowEdge(1), src.GetXaxis().GetBinUpEdge(src.GetNbinsX()),
                   src.GetNbinsY(), src.GetYaxis().GetBinLowEdge(1), src.GetYaxis().GetBinUpEdge(src.GetNbinsY()))
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)
    for ibinz in range(ibinz0, ibinz1+1):
        for ibinx in range(1, src.GetNbinsX()+1):
            for ibiny in range(1, src.GetNbinsY()+1):
                h2.SetBinContent(ibinx,ibiny,h2.GetBinContent(ibinx,ibiny)+src.GetBinContent(ibinx,ibiny,ibinz))

    emcal_pixel_maps.append(h2)

In [19]:
txt_path="output/hot_pixels.txt"
half_win=3
nsig=5.0
min_neighbors=10
min_excess=5.0
zero_cut=0.0
"""
1) Read existing hot map
2) Build masked clones per sector
3) Re-run detection; collect only *new* hot pixels
4) Update TXT (deduped + sorted)
5) Plot masked maps with new hot overlays
"""
# 1) read current map
hot_dict = load_hot_txt(txt_path)
# 2-3) run per sector
c = root.TCanvas("c_masked_hot", "Masked maps + NEW hot pixels", 1450, 900*2)
nsec = len(emcal_pixel_maps)
cols = 3
rows = int(np.ceil(nsec/float(cols)))
c.Divide(cols, rows)
info = []
any_new = False
for s in range(nsec):
    c.cd(1+s)
    masked = masked_clone(emcal_pixel_maps[s], s, hot_dict)
    info.append((s, masked))
    masked.Draw("COLZ")
    g, hot_list = find_hot_pixels(masked, half_win=half_win, nsig=nsig,
                                  min_neighbors=min_neighbors, min_excess=min_excess,
                                  zero_cut=zero_cut, make_zmap=False)
    # Filter out already-known pixels
    new_pairs = []
    known = hot_dict.get(s, set())
    for (ix0, iy0, x, y, cval, med, z) in hot_list:
        iy0_z = iy0
        iz0_z = ix0
        if (iy0_z, iz0_z) not in known:
            new_pairs.append((iy0_z, iz0_z))
    if new_pairs:
        any_new = True
        info.append(g)
        # overlay only the new ones (optional: hide g and draw a reduced graph)
        g.Draw("P SAME")
        # update dict
        hot_dict.setdefault(s, set()).update(new_pairs)
c.Draw()
# 4) write back if needed
if any_new:
    nrows = write_hot_txt(txt_path, hot_dict)
    print(f"Updated {txt_path} with new hot pixels. Total lines: {nrows}")
else:
    print("No new hot pixels found. Map unchanged.")

No new hot pixels found. Map unchanged.


In [14]:
if False:# ----- Example call (after you have emcal_pixel_maps built) -----
    any_new,info = test_and_update_hot_map(
    emcal_pixel_maps,
    txt_path="output/hot_pixels.txt",
    half_win=3, nsig=5.0, min_neighbors=10, min_excess=5.0, zero_cut=0.0
)


In [31]:
old_txt = "input/EmcalDeadMap.txt"  # <-- change to your old map
new_txt = "output/hot_pixels.txt"       # <-- your newly found hot pixels

from collections import defaultdict

def load_map(path):
    """
    Load 'sector iy iz flag' -> dict{(s,iy,iz): flag}.
    If the same key appears multiple times with different flags,
    resolve by taking max(flag) so '1' wins over '0'.
    """
    m, conflicts = {}, []
    with open(path) as f:
        for ln, line in enumerate(f, 1):
            line = line.strip()
            if not line or line.startswith("#"): 
                continue
            parts = line.split()
            if len(parts) < 4:
                print(f"[warn] skip malformed line {ln}: {line}")
                continue
            s, iy, iz, fl = map(int, parts[:4])
            key = (s, iy, iz)
            if key in m and m[key] != fl:
                conflicts.append((ln, key, m[key], fl))
                m[key] = max(m[key], fl)  # prefer '1'
            else:
                m[key] = fl
    return m, conflicts

def write_list(path, items, flag):
    """Write a list of (s,iy,iz) with a fixed flag."""
    with open(path, "w") as f:
        for s, iy, iz in sorted(items):
            f.write(f"{s} {iy} {iz} {flag}\n")

def write_map(path, m):
    """Write the full map dict back to disk (sorted)."""
    with open(path, "w") as f:
        for (s, iy, iz), fl in sorted(m.items()):
            f.write(f"{s} {iy} {iz} {fl}\n")

def by_sector(tuples):
    d = defaultdict(list)
    for s, iy, iz in sorted(tuples):
        d[s].append((iy, iz))
    return d

# ---- load files ----
old_map, old_conf = load_map(old_txt)
new_map, new_conf = load_map(new_txt)
# >>> apply sector reversal to OLD only <<<
#NSEC = 1 + max((s for (s,_,_) in old_map.keys()), default=7)  # will be 8 for 0..7
#old_map = { (NSEC-1 - s, iy, iz): fl for (s,iy,iz), fl in old_map.items() }


if old_conf:
    print(f"[note] {len(old_conf)} conflicting lines in OLD were resolved with '1' winning.")
if new_conf:
    print(f"[note] {len(new_conf)} conflicting lines in NEW were resolved with '1' winning.")

hot_old = {k for k, v in old_map.items() if v == 1}
hot_new = {k for k, v in new_map.items() if v == 1}

newly_hot   = hot_new - hot_old              # became hot now
resolved    = hot_old - hot_new              # were hot before, not hot now
unchanged   = hot_new & hot_old              # stayed hot

# ---- print summaries ----
print(f"\nTotals:")
print(f"  OLD hot: {len(hot_old)}")
print(f"  NEW hot: {len(hot_new)}")
print(f"  Newly hot: {len(newly_hot)}")
print(f"  Resolved (no longer hot): {len(resolved)}")
print(f"  Unchanged hot: {len(unchanged)}")

print("\nPer-sector summary:")
secs = sorted({s for (s,_,_) in hot_old | hot_new})
for s in secs:
    o = sum(1 for k in hot_old if k[0]==s)
    n = sum(1 for k in hot_new if k[0]==s)
    nh = sum(1 for k in newly_hot if k[0]==s)
    rs = sum(1 for k in resolved  if k[0]==s)
    print(f"  sec {s}: old={o:3d}  new={n:3d}  newly_hot={nh:3d}  resolved={rs:3d}")

# ---- (optional) print actual coordinates (small lists) ----
def print_list(title, items, limit=50):
    print(f"\n{title} (count={len(items)})")
    for i, (s, iy, iz) in enumerate(sorted(items)):
        if i >= limit:
            print(f"  ... (+{len(items)-limit} more)")
            break
        print(f"  {s} {iy} {iz} 1")

print_list("Newly hot", newly_hot)
print_list("Resolved (were hot, now not)", resolved)

# ---- (optional) write diffs and merged map ----
#write_list("diff_newly_hot.txt", newly_hot, 1)
#write_list("diff_resolved_hot.txt", resolved, 0)

# merged map = start from OLD, then apply NEW flags (1 wins where NEW says hot)
merged = dict(old_map)
for k, v in new_map.items():
    if v == 1:
        merged[k] = 1
write_map("merged_map.txt", merged)
print("\nWrote: diff_newly_hot.txt, diff_resolved_hot.txt, merged_map.txt")



Totals:
  OLD hot: 576
  NEW hot: 149
  Newly hot: 52
  Resolved (no longer hot): 479
  Unchanged hot: 97

Per-sector summary:
  sec 0: old= 16  new=  1  newly_hot=  0  resolved= 15
  sec 1: old= 11  new=  0  newly_hot=  0  resolved= 11
  sec 2: old= 31  new=  3  newly_hot=  3  resolved= 31
  sec 3: old= 48  new= 21  newly_hot= 13  resolved= 40
  sec 4: old= 20  new=  2  newly_hot=  2  resolved= 20
  sec 5: old= 35  new=  2  newly_hot=  1  resolved= 34
  sec 6: old=169  new= 53  newly_hot= 29  resolved=145
  sec 7: old=246  new= 67  newly_hot=  4  resolved=183

Newly hot (count=52)
  2 35 50 1
  2 35 63 1
  2 35 67 1
  3 0 54 1
  3 30 61 1
  3 32 8 1
  3 32 9 1
  3 32 13 1
  3 33 9 1
  3 33 12 1
  3 34 8 1
  3 34 9 1
  3 35 8 1
  3 35 9 1
  3 35 60 1
  3 35 61 1
  4 15 7 1
  4 23 11 1
  5 29 47 1
  6 0 25 1
  6 0 34 1
  6 0 45 1
  6 3 62 1
  6 3 65 1
  6 4 61 1
  6 6 39 1
  6 6 62 1
  6 7 61 1
  6 11 68 1
  6 16 39 1
  6 23 46 1
  6 25 30 1
  6 27 26 1
  6 27 27 1
  6 28 38 1
  6 31 4

In [32]:
def write_map_like_old(path, m):
    """
    m: dict keyed by (s, iy, iz) -> flag (0/1/2/3)
    Writes lines as: s iy iz flag
    Sorted by (s, iz, iy) to match the old visual order.
    """
    keys = sorted(m.keys(), key=lambda k: (k[0], k[2], k[1]))  # (s, iz, iy)
    with open(path, "w") as f:
        for (s, iy, iz) in keys:
            f.write(f"{s} {iy} {iz} {m[(s, iy, iz)]}\n")


In [33]:
# --- build merged map:
# flags: 0 = normal (from OLD), 1 = hot only in OLD, 2 = newly hot (NEW only), 3 = certainly hot (OLD ∩ NEW)
merged = dict(old_map)   # keep everything from OLD as-is (0/1)

newly_hot_keys = set()
certain_hot_keys = set()

for key, v in new_map.items():
    if v != 1:
        continue  # only care about hot in NEW
    if old_map.get(key, 0) == 1:
        # was already hot before -> mark as 3 (certainly hot)
        merged[key] = 3
        certain_hot_keys.add(key)
    else:
        # not hot before -> mark as 2 (newly hot)
        merged[key] = 2
        newly_hot_keys.add(key)

# --- quick counts
cnt0 = sum(1 for fl in merged.values() if fl == 0)
cnt1 = sum(1 for fl in merged.values() if fl == 1)
cnt2 = sum(1 for fl in merged.values() if fl == 2)
cnt3 = sum(1 for fl in merged.values() if fl == 3)
print(f"\nMerged map counts: flag0={cnt0}  flag1={cnt1}  flag2(newly)= {cnt2}  flag3(certain)= {cnt3}")

# (optional) per-sector summary for 2 and 3
from collections import Counter
c2 = Counter(k[0] for k in newly_hot_keys)
c3 = Counter(k[0] for k in certain_hot_keys)
print("\nPer-sector (flag=2 newly hot / flag=3 certainly hot):")
for s in sorted(set(list(c2.keys()) + list(c3.keys()))):
    print(f"  sec {s}: new2={c2.get(s,0):3d}  certain3={c3.get(s,0):3d}")

# (optional) write lists for quick inspection
# write_list("diff_newly_hot.txt", newly_hot_keys, 2)
# write_list("diff_certainly_hot.txt", certain_hot_keys, 3)

# write merged file (contains 0/1/2/3)
write_map_like_old("merged_map.txt", merged)
print("Wrote merged_map.txt (flags 0/1/2/3)")



Merged map counts: flag0=24140  flag1=479  flag2(newly)= 52  flag3(certain)= 97

Per-sector (flag=2 newly hot / flag=3 certainly hot):
  sec 0: new2=  0  certain3=  1
  sec 2: new2=  3  certain3=  0
  sec 3: new2= 13  certain3=  8
  sec 4: new2=  2  certain3=  0
  sec 5: new2=  1  certain3=  1
  sec 6: new2= 29  certain3= 24
  sec 7: new2=  4  certain3= 63
Wrote merged_map.txt (flags 0/1/2/3)


In [20]:
print(hot_dict)

{0: {(0, 20)}, 2: {(35, 67), (35, 50), (35, 63)}, 3: {(0, 54), (30, 61), (0, 57), (32, 9), (35, 60), (15, 60), (35, 8), (34, 9), (0, 50), (0, 53), (19, 60), (32, 8), (33, 13), (31, 9), (34, 8), (0, 49), (35, 61), (33, 9), (32, 13), (33, 12), (35, 9)}, 4: {(23, 11), (15, 7)}, 5: {(7, 38), (29, 47)}, 6: {(4, 61), (0, 45), (3, 62), (3, 68), (15, 39), (3, 65), (36, 13), (44, 45), (6, 39), (47, 86), (35, 69), (36, 40), (23, 46), (32, 27), (37, 45), (43, 94), (15, 32), (41, 51), (44, 47), (46, 44), (36, 67), (38, 40), (46, 47), (47, 67), (11, 68), (21, 36), (31, 40), (27, 27), (11, 80), (7, 61), (18, 0), (16, 73), (6, 62), (28, 38), (39, 62), (0, 25), (15, 80), (0, 34), (44, 95), (4, 62), (25, 30), (44, 43), (46, 95), (47, 72), (44, 46), (0, 55), (16, 39), (46, 46), (11, 9), (27, 26), (45, 44), (42, 94), (43, 92)}, 7: {(39, 73), (35, 33), (21, 80), (36, 62), (40, 84), (23, 80), (31, 81), (32, 58), (37, 76), (36, 74), (22, 32), (4, 24), (47, 28), (4, 33), (43, 18), (16, 68), (38, 1), (43, 21)

In [38]:
def load_hot_txt_multiflag(path):
    """
    Returns: {1: {sec: set((iy,iz))}, 2: {...}, 3: {...}}
    Ignores flag==0. Works even if the file only has 1s.
    """
    groups = {1: {}, 2: {}, 3: {}}
    if not os.path.exists(path):
        return groups
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            parts = line.split()
            if len(parts) < 4:
                continue
            s, iy, iz, flag = map(int, parts[:4])
            if flag in (1, 2, 3):
                groups.setdefault(flag, {}).setdefault(s, set()).add((iy, iz))
    return groups


In [42]:
hot_groups = load_hot_txt_multiflag(old_txt)  # {1:{sec:set()}, 2:{}, 3:{}}

c = root.TCanvas("c_masked_hot", "Masked maps + OLD hot pixels (1/2/3)", 1450, 900*2)
nsec = len(emcal_pixel_maps_old)
cols = 3
rows = int(np.ceil(nsec/float(cols)))
c.Divide(cols, rows)

rings = []
for s in range(nsec):
    pad = c.cd(1+s)
    h = emcal_pixel_maps_old[s]
    h.Draw("COLZ")

    xax, yax = h.GetXaxis(), h.GetYaxis()

    # (flag, color, linestyle, linewidth)
    styles = [
        (1, root.kRed+1,   1, 2),  # old hot
        (2, root.kMagenta,  2, 2),  # newly hot
        (3, root.kBlack,   1, 3),  # certainly hot (in both)
    ]

    for flag, col, ls, lw in styles:
        for (iy0, iz0) in hot_groups.get(flag, {}).get(s, set()):
            xc = xax.GetBinCenter(iz0+1)
            yc = yax.GetBinCenter(iy0+1)
            rx = 0.5*(xax.GetBinUpEdge(iz0+1) - xax.GetBinLowEdge(iz0+1))
            ry = 0.5*(yax.GetBinUpEdge(iy0+1) - yax.GetBinLowEdge(iy0+1))

            e = root.TEllipse(xc, yc, rx, ry)
            e.SetFillStyle(0)     # no fill
            e.SetLineColor(col)
            e.SetLineStyle(ls)
            e.SetLineWidth(lw)
            e.Draw("SAME")
            rings.append(e)
c.Draw()
